In [1]:
import sgkit as sg
import pandas as pd
import xarray as xr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
def is_nonsynonymous(mutation_string):
    mutation_code = ds_logreg_ftsI.mutation.to_series().str.split().str[0]
    return (mutation_code.str[0] != mutation_code.str[-1]).values

ds_logreg = sg.load_dataset('../results/zarrs/Hinf_norm_bin_logreg_results.zarr/')
ds_logreg_ftsI = ds_logreg.sel(variants=((1688288 < ds_logreg.variant_position) & (ds_logreg.variant_position < 1690120)).compute())
ds_logreg_ftsI = ds_logreg_ftsI.sel(variants=is_nonsynonymous(ds_logreg_ftsI.mutation))

mask = ((ds_logreg_ftsI.call_genotype.loc['ID_1688312_A'] == 1) & (ds_logreg_ftsI.call_genotype.loc['ID_1688338_T'] == 1)).compute().squeeze()
ds_logreg_ftsI.sel(samples=mask).AMP.to_series().value_counts()['S'] 


32

In [ ]:
from itertools import product

def is_nonsynonymous(mutation_string):
    mutation_code = ds_logreg_ftsI.mutation.to_series().str.split().str[0]
    return (mutation_code.str[0] != mutation_code.str[-1]).values

ds_logreg = sg.load_dataset('results/zarrs/Hinf_norm_bin_logreg_results.zarr/')
ds_logreg_ftsI = ds_logreg.sel(variants=((1688288 < ds_logreg.variant_position) & (ds_logreg.variant_position < 1690120)).compute())
ds_logreg_ftsI = ds_logreg_ftsI.sel(variants=is_nonsynonymous(ds_logreg_ftsI.mutation))



variants = ds_logreg_ftsI.variants.values.tolist()

epsilon = 0.00001
ratios = {}
i = 0
for variant1, variant2 in product(variants, variants):
    if variant1 == variant2:
        pass
    else:
        mask = ((ds_logreg_ftsI.call_genotype.loc[variant1] == 1) & (ds_logreg_ftsI.call_genotype.loc[variant2] == 1)).compute().squeeze()
        subset_combi = ds_logreg_ftsI.sel(samples=mask)
        
        has_combi = mask.sum().values
        has_combi_R = subset_combi.AMP.to_series().value_counts().get('R', default=epsilon)
        has_combi_S = subset_combi.AMP.to_series().value_counts().get('S', default=epsilon)
        

        subset_combi_not = ds_logreg_ftsI.sel(samples=~mask)
        has_combi_not = (~mask).sum().values
        has_combi_not_R = subset_combi_not.AMP.to_series().value_counts().get('R', default=epsilon)
        has_combi_not_S = subset_combi_not.AMP.to_series().value_counts().get('S', default=epsilon)

        # print(has_combi, has_combi_not)
        odds_ratio = (has_combi_R/has_combi_S) / (has_combi_not_R/has_combi_not_S)
        ratios[(variant1, variant2)] = odds_ratio
        # print(has_combi_R+has_combi_S+has_combi_not_R+has_combi_not_S)
        # print(variant1, variant2)
        # print(has_combi_R, has_combi_S, has_combi_not_R, has_combi_not_S)
        # print(odds_ratio)
        # print('-------')

    if i == 40:
        break
    i += 1
        

ratios = pd.Series(ratios)


for variant in variants:
    mask = (ds_logreg_ftsI.call_genotype.loc[variant] == 1).values.squeeze()
    # print(mask)
    subset_has_mutation = ds_logreg_ftsI.sel(samples=mask)
    has_mutation_R = subset_has_mutation.AMP.to_series().value_counts().get('R', default=epsilon)
    has_mutation_S = subset_has_mutation.AMP.to_series().value_counts().get('S', default=epsilon)

    subset_has_not_mutation = ds_logreg_ftsI.sel(samples=~mask)
    has_not_mutation_R = subset_has_not_mutation.AMP.to_series().value_counts().get('R', default=epsilon)
    has_not_mutation_S = subset_has_not_mutation.AMP.to_series().value_counts().get('S', default=epsilon)

    odds_ratio = (has_mutation_R / has_mutation_S) / (has_not_mutation_R / has_not_mutation_S)
    print(variant)
    print(ds_logreg_ftsI.mutation.loc[variant].values)
    print(has_mutation_R, has_mutation_S, has_not_mutation_R, has_not_mutation_S)
    print(odds_ratio)
    print('-------------')

In [ ]:
ds_logreg_ftsI.call_genotype.loc['ID_1688990_A'].values

In [ ]:
ds_logreg_ftsI.mutation.loc['ID_1689466_T'].values

In [17]:
ds_linreg = sg.load_dataset('results/zarrs/Hinf_norm_mic_linreg_results.zarr/')
ds_linreg.effect.loc['ID_1688990_A'].values

array(1.03190352)

### Linear regression for riccardo tool

In [14]:
def is_nonsynonymous(mutation_string):
    # A20A (GCC/GCG) -> [A20A, (GCC/GCG)]
    mutation_code = mutation_string.to_series().str.split().str[0]
    # A_1 ?= A_2
    return ((mutation_code.notna()) & (mutation_code.str[0] != mutation_code.str[-1])).values

ds_linreg = sg.load_dataset('../results/zarrs/Hinf_norm_mic_linreg_logscaled_results.zarr/')
ds_linreg_ftsI = ds_linreg.sel(variants=((1688288 < ds_linreg.variant_position) & (ds_linreg.variant_position < 1690120)).compute())
# print(ds_linreg_ftsI.dims)
# print(ds_linreg_ftsI.pipe(sg.variant_stats)['variant_allele_count'].values)
ds_linreg_ftsI = ds_linreg_ftsI.sel(variants=is_nonsynonymous(ds_linreg_ftsI.mutation))
# print(ds_linreg_ftsI.variants)
ds_linreg_ftsI['variant_id'] = ds_linreg_ftsI['mutation']



# the corresponding attributes somehow disappeared, so set them manually
ds_linreg_ftsI.attrs = {'contig_lengths' :[1830701],
                    'contigs' :['Hinf_Rd-KW20v3'],
                    'filters' :['PASS', 'LowQual']}

ds_linreg_ftsI_transpep = ds_linreg_ftsI.sel(variants=((1688542 < ds_linreg_ftsI.variant_position) & (ds_linreg_ftsI.variant_position < 1689052)).compute())
ds_linreg_ftsI_transpep.variants  


<xarray.DataArray 'variants' (variants: 10)>
array(['ID_1688543_T', 'ID_1688543_C', 'ID_1688616_A', 'ID_1688617_T',
       'ID_1688652_T', 'ID_1688770_T', 'ID_1688776_C', 'ID_1688967_G',
       'ID_1688990_A', 'ID_1689051_T'], dtype=object)
Coordinates:
  * variants  (variants) object 'ID_1688543_T' 'ID_1688543_C' ... 'ID_1689051_T'

### Write to zarr and than to vcf

resistance_groups = ['ID_1688990_A', 'ID_1688990_G', 'ID_1688967_G', 'ID_1688954_A', 'ID_1688776_C', 'ID_1688616_A', 'ID_1688617_T', 'ID_1688571_T', 'ID_1688543_T', 'ID_1688543_C']



In [15]:
from sgkit.io.vcf import write_vcf

sg.save_dataset(ds_linreg_ftsI.reset_index('variants').reset_coords('variants'), '../test/linreg_nonsynonymous.zarr', auto_rechunk=True)
# ds_linreg_ftsI.reset_index('variants').reset_coords('variants')
sg.save_dataset(ds_linreg_ftsI_transpep.reset_index('variants').reset_coords('variants'), '../test/linreg_nonsynonymous_transpep.zarr', auto_rechunk=True)


In [16]:
from sgkit.io.vcf import zarr_to_vcf

zarr_to_vcf('../test/linreg_nonsynonymous.zarr', '../test/linreg_nonsynonymous.vcf')
zarr_to_vcf('../test/linreg_nonsynonymous_transpep.zarr', '../test/linreg_nonsynonymous_transpep.vcf')

In [17]:
# ds_linreg_ftsI['serotype'] = ds_linreg_ftsI['serotype'].to_series().replace('', 'NA')
ds_linreg_ftsI[['sample_id', 'serotype']].to_pandas().transpose().to_csv('../test/linreg_serotype.csv', sep='\t', index=False, header=False)
ds_linreg_ftsI[['sample_id', 'origin']].to_pandas().transpose().to_csv('../test/linreg_origin.csv', sep='\t', index=False, header=False)
ds_linreg_ftsI[['sample_id', 'AMP']].to_pandas().transpose().to_csv('../test/linreg_resistance.csv', sep='\t', index=False, header=False)
ds_linreg_ftsI[['sample_id', 'AMP_MIC']].to_pandas().transpose().to_csv('../test/linreg_mic.csv', sep='\t', index=False, header=False)

### Logistic regression for riccardo tool

In [73]:
def is_nonsynonymous(mutation_string):
    # A20A (GCC/GCG) -> [A20A, (GCC/GCG)]
    mutation_code = mutation_string.to_series().str.split().str[0]
    # A_1 ?= A_2
    return ((mutation_code.notna()) & (mutation_code.str[0] != mutation_code.str[-1])).values

ds_logreg = sg.load_dataset('../results/zarrs/Hinf_norm_mic_linreg_logscaled_results.zarr/')
ds_logreg_ftsI = ds_logreg.sel(variants=((1688288 < ds_logreg.variant_position) & (ds_logreg.variant_position < 1690120)).compute())
ds_logreg_ftsI = ds_logreg_ftsI.sel(variants=is_nonsynonymous(ds_logreg_ftsI.mutation))
ds_logreg_ftsI['variant_id'] = ds_logreg_ftsI['mutation']


# the corresponding attributes somehow disappeared, so set them manually
ds_logreg_ftsI.attrs = {'contig_lengths' :[1830701],
                    'contigs' :['Hinf_Rd-KW20v3'],
                    'filters' :['PASS', 'LowQual']}
ds_logreg_ftsI_transpep = ds_logreg_ftsI.sel(variants=((1688530 < ds_logreg_ftsI.variant_position) & (ds_logreg_ftsI.variant_position < 1689074)).compute())

ds_logreg_ftsI_transpep.mutation.values

array(['A530S (GCA/TCA)', 'N526K (AAT/AAA)', 'N526K (AAT/AAG)',
       'A502V (GCA/GTA)', 'A502T (GCA/ACA)', 'G490E (GGA/GAA)',
       'R451S (CGT/AGT)', 'I449V (ATT/GTT)', 'S385T (AGT/ACT)',
       'M377I (ATG/ATT)', 'S357N (AGC/AAC)', 'D350N (GAT/AAT)'],
      dtype=object)

In [67]:
mapping = pd.read_csv('../results/mapping.csv')
mapping[mapping.mutation.str.startswith('D350N')]

,Unnamed: 0,position,allele,mutation,id
79962,924521,1689073,T,D350N (GAT/AAT),ID_1689073_T
155982,7380387,696387,T,D350N (GAC/AAC),ID_696387_T
186269,23145735,747722,T,D350N (GAT/AAT),ID_747722_T


### Write to zarr and than to vcf

resistance_groups = ['ID_1688990_A', 'ID_1688990_G', 'ID_1688967_G', 'ID_1688954_A', 'ID_1688776_C', 'ID_1688616_A', 'ID_1688617_T', 'ID_1688571_T', 'ID_1688543_T', 'ID_1688543_C']



In [75]:
from sgkit.io.vcf import write_vcf

sg.save_dataset(ds_logreg_ftsI.reset_index('variants').reset_coords('variants'), '../test/linreg_nonsynonymous.zarr', auto_rechunk=True)
# ds_linreg_ftsI.reset_index('variants').reset_coords('variants')
sg.save_dataset(ds_logreg_ftsI_transpep.reset_index('variants').reset_coords('variants'), '../test/linreg_nonsynonymous_transpep.zarr', auto_rechunk=True)


In [77]:
from sgkit.io.vcf import zarr_to_vcf

zarr_to_vcf('../test/linreg_nonsynonymous.zarr', '../test/linreg_nonsynonymous.vcf')
zarr_to_vcf('../test/linreg_nonsynonymous_transpep.zarr', '../test/linreg_nonsynonymous_transpep.vcf')

In [21]:
# ds_linreg_ftsI['serotype'] = ds_linreg_ftsI['serotype'].to_series().replace('', 'NA')
ds_logreg_ftsI[['sample_id', 'serotype']].to_pandas().transpose().to_csv('../test/logreg_serotype.csv', sep='\t', index=False, header=False)
ds_logreg_ftsI[['sample_id', 'origin']].to_pandas().transpose().to_csv('../test/logreg_origin.csv', sep='\t', index=False, header=False)
ds_logreg_ftsI[['sample_id', 'AMP']].to_pandas().transpose().to_csv('../test/logreg_resistance.csv', sep='\t', index=False, header=False)
ds_logreg_ftsI[['sample_id', 'AMP_MIC']].to_pandas().transpose().to_csv('../test/logreg_mic.csv', sep='\t', index=False, header=False)

### scratch

In [87]:
lin = ds_linreg_ftsI.variants.to_numpy()
log = ds_logreg_ftsI.variants.to_numpy()

np.intersect1d(lin, log)


array(['ID_1688312_A', 'ID_1688314_T', 'ID_1688320_C', 'ID_1688338_T',
       'ID_1688339_T', 'ID_1688340_G', 'ID_1688350_C', 'ID_1688354_T',
       'ID_1688361_A', 'ID_1688365_A', 'ID_1688365_G', 'ID_1688415_C',
       'ID_1688482_T', 'ID_1688533_A', 'ID_1688543_C', 'ID_1688543_T',
       'ID_1688616_A', 'ID_1688617_T', 'ID_1688652_T', 'ID_1688770_T',
       'ID_1688776_C', 'ID_1688967_G', 'ID_1688990_A', 'ID_1689051_T',
       'ID_1689405_T', 'ID_1689438_A', 'ID_1689466_T', 'ID_1689667_C',
       'ID_1689700_T', 'ID_1689747_A', 'ID_1689751_T', 'ID_1690021_G',
       'ID_1690057_T', 'ID_1690102_G'], dtype=object)

In [84]:
log

array(['ID_1688312_A', 'ID_1688314_T', 'ID_1688320_C', 'ID_1688338_T',
       'ID_1688339_T', 'ID_1688340_G', 'ID_1688350_C', 'ID_1688354_T',
       'ID_1688361_A', 'ID_1688365_A', 'ID_1688365_G', 'ID_1688415_C',
       'ID_1688482_T', 'ID_1688533_A', 'ID_1688543_T', 'ID_1688543_C',
       'ID_1688616_A', 'ID_1688617_T', 'ID_1688652_T', 'ID_1688770_T',
       'ID_1688776_C', 'ID_1688967_G', 'ID_1688990_A', 'ID_1689051_T',
       'ID_1689405_T', 'ID_1689438_A', 'ID_1689466_T', 'ID_1689667_C',
       'ID_1689700_T', 'ID_1689747_A', 'ID_1689751_T', 'ID_1690021_G',
       'ID_1690057_T', 'ID_1690102_G'], dtype=object)

In [93]:
ds_linreg_ftsI['mutation'].loc['ID_1688415_C'].values

array('N569S (AAC/AGC)', dtype='<U15')

## Scratch

In [ ]:
def is_nonsynonymous(mutation_string):
    # A20A (GCC/GCG) -> [A20A, (GCC/GCG)]
    mutation_code = mutation_string.to_series().str.split().str[0]
    # A_1 ?= A_2
    return ((mutation_code.notna()) & (mutation_code.str[0] != mutation_code.str[-1])).values

ds_mutation_mapping = (pd.read_csv('../results/mapping.csv').rename(columns={'id' : 'variant_id'}).drop(columns=['Unnamed: 0', 'position', 'allele'])
                       .to_xarray().reset_index('index').reset_coords('index').rename_dims({'index' : 'variants'}).drop_vars('index').set_coords('variant_id').set_index({'variants' : 'variant_id'}))
                       

ds_ds = (
    sg.load_dataset('../results/zarrs/Hinf_norm_mic.zarr/')
    .assign(variant_id=lambda ds: ( 'ID_' + ds.variant_position.to_series().astype(str) + '_' + ds.variant_allele[:,1].to_series()))
    .set_coords('variant_id')
    .set_index({'variants': 'variant_id'})
)
ds_ds = ds_ds.sel(variants=((1688288 < ds_ds.variant_position) & (ds_ds.variant_position < 1690120)).compute())

# df_mutation_mapping['ID']
# ds_mutation_mapping
# ds_ds
ds_ds = ds_ds.merge(ds_mutation_mapping, join='left')
ds_ds

In [306]:
ds_ds = (ds_ds.drop_vars(['call_genotype_mask', 'call_genotype_phased', 'variant_id_mask'])
            .assign(call_genotype = lambda ds: ds.call_genotype.max(dim='ploidy'))
)

ds_ds['call_genotype'] = ds_ds.call_genotype.expand_dims({'ploidy':1},2)
ds_ds['call_genotype_mask'] = xr.zeros_like(ds_ds.call_genotype).astype(bool)

ds_ds = (ds_ds
         .pipe(sg.variant_stats)
         .assign(variant_ref= lambda ds: ds.variant_allele[:,0],
                 variant_alt= lambda ds: ds.variant_allele[:,1],
                 variant_ref_count=lambda ds: ds.variant_allele_count[:,0],
                variant_alt_count=lambda ds: ds.variant_allele_count[:,1])
)

ds_ds[['variants', 'mutation', 'variant_ref', 'variant_alt', 'variant_ref_count', 'variant_alt_count']].to_dataframe()

,mutation,variant_ref,variant_alt,variant_ref_count,variant_alt_count
variants,,,,,
ID_1688289_C,NaN,T,C,265,0
ID_1688296_T,V609M (GTG/ATG),C,T,264,1
ID_1688309_A,H604H (CAC/CAT),G,A,260,5
ID_1688312_A,E603D (GAA/GAT),T,A,154,111
ID_1688312_C,E603E (GAA/GAG),T,C,264,1
...,...,...,...,...,...
ID_1690103_A,S6S (TCC/TCT),G,A,264,1
ID_1690109_G,F4F (TTT/TTC),A,G,264,1
ID_1690112_C,K3K (AAA/AAG),T,C,264,1


In [221]:
df_mutations_ftsI = ds_ds.pipe(sg.variant_stats)[['mutation']].to_pandas()
mask = df_mutations_ftsI['mutation'].str.startswith('R517', na=False)
df_mutations_ftsI[mask]

,mutation
variants,
ID_1688571_T,R517H (CGT/CAT)


In [ ]:
ID_1688954_A
ID_1688571_T

In [ ]:
ds_bin = sg.load_dataset('../results/zarrs/Hinf_norm_mic.zarr/').assign(variant_id=lambda ds: ( 'ID_' + ds.variant_position.to_series().astype(str) + '_' + ds.variant_allele[:,1].to_series())).set_coords('variant_id').set_index({'variants': 'variant_id'})
# ds_bin.pipe(sg.variant_stats).sel(variants=ds_bin.variants == 'ID_1688571_T').variant_allele_count.values

In [263]:
265 * 2
514 + 16
626 + 18

644

In [ ]:
sg.load_dataset('../results/zarrs/Hinf_norm_bin_logreg_results.zarr/').sel(variants=( lambda ds: ds.variants == 'ID_1690119_A'))